<a href="https://colab.research.google.com/github/hikmahealth/covid19countymap/blob/master/Hikma_COVID_19_Native_Community_Google_Sheet_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hikma COVID-19 Response - County Google Sheet Export

License: Apache 2.0

In [0]:
import copy
import io
import json
import urllib

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd
import numpy as np


## Use census data to map county name to FIPS code

In [0]:
with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))

In [0]:
census = raw_census[raw_census.COUNTY != 0].copy()

In [0]:
census["fips"] = census.STATE * 1000 + census.COUNTY
census.set_index("fips", inplace=True)
census["full_name"] = census.CTYNAME + ", " + census.STNAME
fips_code_mapping = {name: i for i, name in census.full_name.items()}

## Read the data from Google Sheets

In [0]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1LRqAKVYpa2uhQcwD8rragwuvU6gMLRzYupihsURC8Cg/edit#gid=519201207")

In [0]:
raw_policies = pd.DataFrame(sheet.sheet1.get_all_records())

## Clean up the county-level data

### Encode free-text column values

In [0]:
dimensions = ["school", "work", "shelter", "shelter_enforcement", "event", "testing", "transport"]

In [0]:
full_policies = raw_policies.rename(columns={
    "You are filling out the form for the following county:": "county_name",
    "Are schools closed in this county?": "school",
    "If yes, please provide the URL to information regarding school closures.": "school_url",
    "If yes, please provide the date of when this policy on school closures took or will take effect.": "school_date",
    "Are non-essential workplaces closed? Is there any language that suggests that those who do not provide essential services should work from home or stop working?": "work",
    "If yes, please provide the URL to information regarding non-essential workplace closures.": "work_url",
    "If yes, provide the date of when this policy took or will take effect.": "work_date",
    "Is social distancing or shelter-in-place being enforced with fines or penalties in this county? ": "shelter_enforcement",
    "If yes, please provide the URL to information regarding fine or penalty enforcement.": "shelter_enforcement_url",
    "If yes, please provide the date of when this fine or penalty enforcement took or will take effect.": "shelter_enforcement_date",
    "Is there a shelter-in-place or stay-at-home order in this county? For example, were people advised to maintain social distance or abstain from all physical interactions or non-essential trips whenever possible?": "shelter",
    "If yes, please provide the URL to information regarding the shelter-in-place or social distancing policy.": "shelter_url",
    "If yes, please provide the date of when this shelter-in-place or social distancing policy took or will take effect.": "shelter_date",
    "Is there any order that public events should be restricted in size or outright cancelled?": "event",
    "If yes, please provide the URL to information regarding public events restrictions.": "event_url",
    "If yes, please provide the date of when the cancellation of public events was or will be in effect.": "event_date",
    "Are there public facilities with COVID testing available in this county? ": "testing",
    "If yes, please provide the URL to information regarding testing facilities.": "testing_url",
    "If yes, please provide the date of when these facilities became available.": "testing_date",
    "Look for any information about whether this county's  buses, subways, light rail, etc. are being shut down. Is any public transit system shut down?": "transport",
    "If yes, please provide the URL to information regarding public transit.": "transport_url",
    "If yes, please provide the date of when public transit was or will be stopped.": "transport_date",
})
full_policies.pop("Are there travel restrictions in this county? For example, are people required to self-quarantine when arriving into this county?")
full_policies.pop("Are airports closed in this county? ")
full_policies["fips"] = full_policies.county_name.map(fips_code_mapping)
full_policies.set_index("fips", inplace=True)
full_policies["updated"] = pd.to_datetime(full_policies.pop("Timestamp"))
for col in dimensions:
  full_policies[col] = full_policies[col].map({"Yes": True, "No": False, "":float('nan')})
# Convert timestamp from ambiguous DD/MM/YYYY to ISO 8601 friendly YYYY-MM-DD
for col in dimensions:
  full_policies[col + "_date"] = pd.to_datetime(full_policies[col + "_date"]).dt.strftime("%Y-%m-%d")

In [0]:
full_policies.school.unique()

array([True, False, nan], dtype=object)

### Deduplicate rows.

Naive assumption - we assume that restrictions are never lifted, so the heaviest restriction reported for fips is still active.

We will likely have to revisit this in the future.

In [0]:
# JS map not aligned to 'City' or 'Municipality' for now 
# policies_noloc = full_policies[~full_policies.county_name.str.contains('city') & ~full_policies.county_name.str.contains('Municipality')].copy()
# JS map can handle the data with missing FIPS codes, so including other non-county locations for now
# policies_noloc = full_policies
# Only deduplicating and keeping first entry for now, while debugging groupby
# policies_min = full_policies.groupby(by="fips").agg(func=max)
# policies_min = full_policies.groupby('fips').min().reset_index()

policies_dedup = full_policies.drop_duplicates(subset="county_name").reset_index()

# Remove if NA for FIPS (if submitter of data changed county name and FIPS could not be correctly mapped
policies_min = policies_dedup[~policies_dedup.fips.isna() & (policies_dedup.fips < 100000) & (policies_dedup.fips >= 0)].copy()

policies_min2 = policies_min

In [0]:
policies_min2.set_index('fips', inplace=True);
# full_policies.groupby('fips')['updated'].sum()

In [0]:
# function that has the following aggregation behavior:
# pick the most recently submitted entry on any particular policy, inherit latest policy for each policy 
# EXCEPT if 2 or more within 24 hours of the last entered policy
# in the case of 2 entries with disagreement, pick the one that is most open (i.e. favor No over Yes)
# if 3+ entries within 24 hours then take majority vote, leaning No in case of tie

def agg24(df):
  polcols = dimensions
  # one entry should be returned for each county, initialize to first entry and edit later
  findf = df.head(1)
  for col in polcols:
    # find last non-null datapoint for that policy
    filldf = df[~df[col].isnull()]
    tmax = filldf.updated.max()
    tcut = tmax - np.timedelta64(24,'h')
    reduced = filldf.drop(filldf[filldf.updated < tcut].index)

    # will return date and URL of the last entry that matched the final boolean
    urlcol = col + '_url'
    datecol = col + '_date'
    
    if reduced[col].size == 0:
       decision = float('nan')
    elif np.sum(reduced[col])/reduced[col].size > 0.50001:
      decision = True
    else:
      decision = False
    findf[col].iloc[0] = decision
    sourced = reduced[reduced[col] == decision]

    if sourced[col].shape[0] > 0:
      # print(sourced.shape[0])
      findf[urlcol].iloc[0] = sourced[urlcol].iloc[sourced.shape[0]-1]
      findf[datecol].iloc[0] = sourced[datecol].iloc[sourced.shape[0]-1]

 #   if reduced[col].isna().all()
  return findf
  

In [0]:
policies_agg = full_policies.groupby('fips').apply(agg24)

In [0]:
policies_agg = policies_agg.droplevel(1)

In [0]:
policies_agg.pop("county_name");
# policies_min.set_index('fips', inplace=True);
#Identifying the problematic row
# policies_debug = policies_min[0:140]
# policies_min = policies_debug

In [0]:
with open("county_policies.json", "w") as outfile:
  outfile.write(policies_agg.to_json(orient="index"))

In [0]:
with open("county_policies.csv", "w") as outfile:
  outfile.write(policies_agg.to_csv())

In [0]:
policies_agg.head()

,school,school_url,school_date,work,work_url,work_date,shelter_enforcement,shelter_enforcement_url,shelter_enforcement_date,shelter,shelter_url,shelter_date,event,event_url,event_date,testing,testing_url,testing_date,transport,transport_url,transport_date,updated
fips,,,,,,,,,,,,,,,,,,,,,,
1001,True,https://whnt.com/news/coronavirus/state-of-eme...,2020-03-18,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alabamapublichealth.gov/news/2020/...,2020-05-01,NaN,,NaN,2020-05-10 02:47:49
1003,True,https://www.bcbe.org/Page/22126,2020-03-21,True,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,False,,NaN,True,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://alpublichealth.maps.arcgis.com/apps/op...,2020-04-23,False,,NaN,2020-05-18 23:39:05
1005,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://alabamapublichealth.gov/legal/assets/o...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.mainstreetfamilycare.com/coronavir...,NaN,False,,NaN,2020-05-23 18:51:26
1007,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.alabamapublichealth.gov/legal/asse...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,True,http://alabamapublichealth.gov/news/2020/05/11...,2020-05-11,False,,NaN,2020-05-23 19:22:13
1009,True,https://whnt.com/news/covid-19/covid-19-impact...,2020-03-16,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alabamapublichealth.gov/news/2020/...,2020-04-22,NaN,,NaN,2020-05-10 02:14:32


In [0]:
# Reformatting for Tableau
# policies_min2['county_only'] = policies_min2['county_name'].str.split(', ', expand=True)[0]
# policies_min2['state_only'] = policies_min2['county_name'].str.split(', ', expand=True)[1]

In [0]:
# policies_min2.head()

In [0]:
# policies_min.columns

In [0]:
# cols = ['county_only','state_only']
# cols = [*cols, *policies_min.columns]

In [0]:
# policies_min2 = policies_min2[cols]

In [0]:
# policies_min2.head()

In [0]:
# with open("county_policies_2.csv", "w") as outfile:
  # outfile.write(policies_min2.to_csv())